# 実験として保存する

現在の実験記録を実験リポジトリとしてデータガバナンス機能に保存し、この環境で次の実験を行えるよう準備します。  

## 1. 保存先の実験リポジトリを作成する

保存したい実験の実験リポジトリ名を入力してください。

In [ ]:
import re
from IPython.display import clear_output

# GINサーバのものに合わせたバリデーションルールを設定
validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

print('作成したい実験リポジトリ名を半角英数字で入力してください。')
while True:
    repository_title = input("リポジトリ名：")
    if validation.fullmatch(repository_title):
        break
    else:
        clear_output()
        print('リポジトリ名は英数字、および"-", "_", "."のみで入力してください。')

clear_output()
print("作成したリポジトリ名：", repository_title)
print('この実験名で処理を進めます。変更したい場合は、このセルをもう一度実行することで訂正ができます。')

In [ ]:
import json
import os

%cd ~/EX-WORKFLOW
from util.scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
%cd ~/
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
# リポジトリ作成のためのユーザ認証
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

In [ ]:
# GIN APIでリポジトリを作成する
import requests

create_repo = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/repos?token=' + tokens[0]['sha1'],
                data={
                    "name": repository_title,
                    "auto_init": False
                }
                )
print(create_repo)

In [ ]:
# 作成した実験リポジトリのリモートURLを作成する
remote_url = params['siblings']['ginSsh']+name+'/'+repository_title+'.git'

# 作成したURLをリモートに登録する
%cd ~/
!datalad siblings add -s $repository_title --url $remote_url

## 2. 実行環境構成を記録する

In [ ]:
%%sh
conda env export -n base > ~/environment.yml
pip freeze > ~/requirements.txt

## 3. 実験記録をデータガバナンス機能に同期する

実験記録をデータガバナンス機能に同期します。  
以下のセルを実行してください。

※データの保存先としてAWS S3準拠のオブジェクトストレージを利用する場合は、  
[こちら](util/operate_S3_annex.ipynb)も併せて実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

# Git管理のパスのリストを作成する
%cd ~/
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
dirs.remove('research_project_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = files

# Git-annex管理するパスの配列を作成する
annexed_save_path = ['input_data', 'output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = repository_title + ' 実験終了 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False, SIBLING_NAME = repository_title)
)

In [ ]:
import papermill as pm

%cd ~/
# Git管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = repository_title + ' 実験終了 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False, SIBLING_NAME = repository_title)
)

## 3. パッケージを初期化して次実験を用意する

In [ ]:
ex_env_title = !git config --get remote.origin.url
ex_env_title = ex_env_title[0].split('/')[-1].replace('.git','')

# 実験フォルダと実験データ格納先を用意する
%cd ~/research_project_data/$ex_env_title
!mkdir -p $repository_title
%cd $repository_title
!mkdir -pv {input_data,output_data,source}

# パッケージフォルダにある現在の実験データを実験フォルダ内に移動させる
%cd ~/research_project_data/$ex_env_title
!mv package/input_data/* $repository_title/input_data/
!mv package/output_data/* $repository_title/output_data/
!mv package/source/* $repository_title/source/

# main.ipynbをsourceフォルダ内に作成する
%cd ~/research_project_data/$ex_env_title/package/source
import nbformat
nb = nbformat.v4.new_notebook()
with open("main.ipynb", "w") as f:
    nbformat.write(nb, f)

## 4. [1. 実験準備]に戻って次の実験を行う